In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import time
import datetime
from sklearn.metrics import mean_absolute_error, mean_squared_error
# Parameter windowing
input_width = 60*2
label_width = 60
shift = label_width
total_window_size = input_width + shift
OUT_STEPS = label_width
# Definisikan irisan untuk input dan label
input_slice = slice(0, input_width)
label_start = total_window_size - label_width
labels_slice = slice(label_start, None)
train_df = None


In [2]:
def  import_data(file_path='D:\\KULIAH\\teep\AI\\dataset\\52 cells\\CellReports.csv'):
    
    df_begin=pd.read_csv(file_path)
    df_1=df_begin.copy()
    df_1.drop(columns=['RRU.PrbAvailDl', 'RRU.PrbAvailUl','RRU.PrbTotUl','RRU.PrbTotDl','RRC.ConnMax'], inplace=True)
    convert_time=pd.to_datetime(df_1['timestamp'], unit='s',origin='unix')
    df_1.insert(df_1.columns.get_loc('timestamp') + 1, 'datetime_column', convert_time)
    df_1.set_index('datetime_column', inplace=True)
    df_1.drop(columns=['timestamp'], inplace=True)
    
    corr_matrix = df_1.corr(numeric_only=True)[['RRU.PrbUsedDl', 'RRU.PrbUsedUl']]
    filtered_corr = corr_matrix[['RRU.PrbUsedDl', 'RRU.PrbUsedUl']]

    # Find columns where correlation with 'RRU.PrbUsedDl' or 'RRU.PrbUsedUl' is greater than 0.5
    columns_with_high_corr = filtered_corr.index[(filtered_corr['RRU.PrbUsedDl'] > 0.5) & (filtered_corr['RRU.PrbUsedUl'] > 0.5)].tolist()

    seleted_columns = ['Viavi.Cell.Name']+ columns_with_high_corr 
    df_2= df_1[seleted_columns].copy()
    cell_name= df_2['Viavi.Cell.Name'].unique()
    df= df_2.copy()
    
    return df, cell_name

In [3]:
def standardize_data(data, train_df, isoutput=True, column_output=['RRU.PrbUsedDl', 'RRU.PrbUsedUl']):
    if isoutput:
        mean = train_df[column_output].mean().values
        std = train_df[column_output].std().values
    else:
        mean = train_df.mean().values
        std = train_df.std().values

    # Reshape for broadcasting with 2D matrix
    mean = mean.reshape(1, -1)
    std = std.reshape(1, -1)

    return (data - mean) / std

def inverse_standardize_data(data, train_df, isoutput=True, column_output=['RRU.PrbUsedDl', 'RRU.PrbUsedUl']):
    if isoutput:
        mean = train_df[column_output].mean().values
        std = train_df[column_output].std().values
    else:
        mean = train_df.mean().values
        std = train_df.std().values

    # Reshape for broadcasting with 2D matrix
    mean = mean.reshape(1, -1)
    std = std.reshape(1, -1)

    return (data * std) + mean


In [4]:
def make_windows(data_x,data_y, total_window_size, input_slice, labels_slice):
    x = []
    y = []
    for i in range(len(data_x) - total_window_size + 1):
        window_x = data_x[i:i+total_window_size]
        x.append(window_x[input_slice])

    for i in range(len(data_y) - total_window_size + 1):
        window_y= data_y[i:i+total_window_size]
        y.append(window_y[labels_slice])

    return np.array(x), np.array(y)


In [5]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import time

def tensorflow_cnn(X_train_scaled, Y_train_scaled, X_validation_scaled, Y_validation_scaled, 
                    learning_rate, target_error, max_epochs, max_sampel_batch,
                    patience, save_best_model_path, validation_data=False, load_model=None, out_steps=OUT_STEPS):
    global model

    class MAEStopCallback(tf.keras.callbacks.Callback):
        def __init__(self, threshold):
            super(MAEStopCallback, self).__init__()
            self.threshold = threshold

        def on_epoch_end(self, epoch, logs=None):
            if logs['mae'] < self.threshold:
                print(f"\nMAE reached below {self.threshold}. Stopping training.")
                self.model.stop_training = True

                
    model = tf.keras.models.Sequential()
    if load_model is None:
        print("use transfer learning")
        return 
    else:
        model = tf.keras.models.load_model(load_model)
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mse', 'mae', 'mape', tf.keras.metrics.RootMeanSquaredError(name='rmse')])
    mae_stop_callback = MAEStopCallback(threshold=target_error)

    checkpoint_callback = ModelCheckpoint(
        save_best_model_path,
        monitor='loss',     
        mode='min',         
        save_best_only=True, 
        verbose=1            
    )

    early_stopping_callback = EarlyStopping(
        monitor='loss',     
        mode='min',         
        patience=patience,    
        restore_best_weights=True,
        verbose=1            
    )

    reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=int(2), min_lr=0.00001, verbose=1)

    time_start = time.time()
    if validation_data:
        model.fit(X_train_scaled, Y_train_scaled, epochs=max_epochs, batch_size=max_sampel_batch,  
                  callbacks=[mae_stop_callback, checkpoint_callback, early_stopping_callback, reduce_lr], 
                  validation_data=(X_validation_scaled, Y_validation_scaled), validation_batch_size=max_sampel_batch)
    else:
        model.fit(X_train_scaled, Y_train_scaled, epochs=max_epochs, batch_size=max_sampel_batch, 
                  callbacks=[mae_stop_callback, checkpoint_callback, early_stopping_callback, reduce_lr])
    
    print("time computation seconds: ", time.time() - time_start)
    
    loss, MSE, MAE, RMSE, MAPE = model.evaluate(X_train_scaled, Y_train_scaled)
    print("loss: ", loss, "MSE: ", MSE, "MAE: ", MAE, "RMSE: ", RMSE, "MAPE: ", MAPE)
    
    return model, loss, MSE, MAE, RMSE, MAPE


In [6]:
def compute_metrics(actual_df, predictions_df):
    mae = mean_absolute_error(actual_df, predictions_df)
    mse = mean_squared_error(actual_df, predictions_df)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((actual_df - predictions_df) / actual_df)) * 100
    return mae, mape, mse, rmse

def process_predictions(model, x_scaled, actual_df, input_width, label_width, columns):
    predictions = model.predict(x_scaled[slice(None, None, label_width), :, :])
    predictions_reshaped = predictions.reshape(-1, len(columns))
    predictions_unscaled = inverse_standardize_data(predictions_reshaped, isoutput=True, train_df=train_df)
    predictions_df = pd.DataFrame(predictions_unscaled, columns=columns, index=actual_df[input_width:].index)
    return predictions_df
def compute_error( x_scaled, actual_df, columns, model):
    
    # Process predictions for training and validation sets
    predictions_df = process_predictions(model, x_scaled, actual_df, input_width, label_width, columns)

    display(pd.concat((predictions_df.rename(columns={columns[0]: '%s_predict'%columns[0], columns[1]: '%s_predict'%columns[1]}), actual_df[input_width:]), axis=1))
    mae, mape, mse, rmse = compute_metrics(actual_df[input_width:], predictions_df)
    
    print(f"MAE: {mae}, MAPE %: {mape}, MSE: {mse}, RMSE: {rmse}")

In [7]:

def plot_predictions(val_df, x_val_scaled, model, val_scaled, input_width, label_width, num_features):
    num_output = 2


    # Predictions for validation data
    val_predictions = model.predict(x_val_scaled[slice(None,None,label_width), :, :])
    predictions_reshaped = val_predictions.reshape(-1, num_output)
    predictions_unscaled = inverse_standardize_data(predictions_reshaped, isoutput=True, train_df=train_df)
    val_predictions_df = pd.DataFrame(predictions_unscaled, columns=['RRU.PrbUsedDl', 'RRU.PrbUsedUl'], index=val_df.index)

    # Generate new predictions
    val_new = val_scaled.tail(input_width).to_numpy()
    val_new = val_new.reshape(1, -1, num_features)
    generate_predictions = model.predict(val_new)
    generate_predictions = inverse_standardize_data(generate_predictions.reshape(-1, num_output), isoutput=True, train_df=train_df)
    generate_predictions_df = pd.DataFrame(generate_predictions, columns=['RRU.PrbUsedDl', 'RRU.PrbUsedUl'], index=val_df.tail(label_width).index + pd.DateOffset(hours=1))

    # Combine predictions
    all_predictions = pd.concat([val_predictions_df, generate_predictions_df], axis=0)
    inputs_graph = val_df[['RRU.PrbUsedDl', 'RRU.PrbUsedUl']].iloc[-label_width * 3:-label_width]
    labels_graph = val_df[['RRU.PrbUsedDl', 'RRU.PrbUsedUl']].iloc[-label_width:]
    predictions_graph = all_predictions.loc[labels_graph.index]
    new_predictions_graph = all_predictions.loc[generate_predictions_df.index]

    # Plotting
    plt.figure(figsize=(16, 8))
    for n, feature in enumerate(all_predictions.columns):
        plt.subplot(len(all_predictions.columns), 1, n + 1)
        plt.plot(inputs_graph.index, inputs_graph[feature], label='Inputs for orange', marker='.', zorder=-100, markersize=5)
        plt.plot(labels_graph.index, labels_graph[feature], label='actual output for orange and inputs for red', marker='.', zorder=-100, c='#2ca02c')
        plt.scatter(predictions_graph.index, predictions_graph[feature], marker='X', edgecolors='k', label='Prediction', c='#ff7f0e', s=int(32 * 1.5))
        plt.scatter(new_predictions_graph.index, new_predictions_graph[feature], marker='+', label='New prediction', c='#FF012D', s=int(32 * 1.5))
        plt.ylabel(feature)
        plt.legend()

    plt.subplots_adjust(hspace=0.2, top=1)
    plt.show()


In [8]:
def running_program(my_df,cell_name, index_cell, name_file, name_file_before):
    global train_df
    df_1= my_df[my_df['Viavi.Cell.Name']==cell_name[index_cell]].drop(columns=['Viavi.Cell.Name']).copy()
    df=df_1.astype(float).copy()
    n = len(df)
    train_df =df.loc[:'2023-01-07 00:00:00'].iloc[:-1]
    val_df = df['2023-01-07 00:00:00':]
    num_features = df.shape[1]
    train_scaled = standardize_data(train_df, isoutput=False, train_df=train_df)
    val_scaled= standardize_data(val_df, isoutput=False, train_df=train_df)
    # Membuat windowed dataset untuk set pelatihan, validasi, dan pengujian

    x_train_scaled, y_train_scaled = make_windows(train_scaled.to_numpy(), train_scaled[['RRU.PrbUsedDl', 'RRU.PrbUsedUl']].to_numpy(),total_window_size, input_slice, labels_slice)
    x_val_scaled, y_val_scaled = make_windows(val_scaled.to_numpy(), val_scaled[['RRU.PrbUsedDl', 'RRU.PrbUsedUl']].to_numpy(),total_window_size, input_slice, labels_slice)

    if index_cell==0:

        model, loss, MSE, MAE, RMSE,  MAPE  = tensorflow_cnn(x_train_scaled, y_train_scaled, x_val_scaled, y_val_scaled,
                                                        learning_rate=0.001, target_error=0.001,  max_epochs=15, max_sampel_batch=60, 
                                                        patience=6,  save_best_model_path = name_file, 
                                                        validation_data=True, load_model='2c_20240721_1610.hdf5', out_steps=OUT_STEPS)
    else:
        print('file before: ', name_file_before)
        model, loss, MSE, MAE, RMSE,  MAPE  = tensorflow_cnn(x_train_scaled, y_train_scaled, x_val_scaled, y_val_scaled,
                                                        learning_rate=0.001, target_error=0.001,  max_epochs=15, max_sampel_batch=60, 
                                                        patience=6,  save_best_model_path = name_file, 
                                                        validation_data=True, load_model=name_file_before, out_steps=OUT_STEPS)
    
    model = tf.keras.models.load_model('%s'%name_file)
    columns = ['RRU.PrbUsedDl', 'RRU.PrbUsedUl']
    output_actual_train = train_df[columns]
    output_actual_val = val_df[columns]
    print("Training Metrics:")
    compute_error(x_train_scaled,  output_actual_train,columns, model)
    print("\nValidation Metrics:")
    compute_error(x_val_scaled,  output_actual_val,columns, model)
    #print("Training Plot:")
    #plot_predictions( output_actual_train[input_width:], x_train_scaled, model, train_scaled, input_width, label_width, num_features)
    #print("\nValidation (test) Plot:")
    #plot_predictions( output_actual_val[input_width:], x_val_scaled, model, val_scaled, input_width, label_width, num_features)
    


In [9]:
df_start, cell_name= import_data(file_path='D:\\KULIAH\\teep\AI\\dataset\\52 cells\\CellReports.csv')
now = datetime.datetime.now()
timestamp = now.strftime("%Y%m%d_%H%M")
for index in range(0,52):
    cells_name= df_start['Viavi.Cell.Name'].unique()
    df_filtered = df_start[df_start['Viavi.Cell.Name'] == cells_name[index]]
    df_no_duplicates = df_filtered.loc[~df_filtered.index.duplicated()]
    name_file='1b_%s_%s.hdf5'%(timestamp, index+1)
    name_file_before='1b_%s_%s.hdf5'%(timestamp, index)
    print(name_file_before)
    print("Cell Name: ", cell_name[index])
    running_program(my_df=df_no_duplicates, cell_name=cell_name, index_cell=index, name_file=name_file, name_file_before=name_file_before)

1b_20240723_1940_0.hdf5
Cell Name:  S1/B2/C1
Epoch 1/15
150/150 [==============================] - ETA: 0s - loss: 0.2949 - mse: 0.2949 - mae: 0.3689 - mape: 212.9063 - rmse: 0.5430
Epoch 1: loss improved from inf to 0.29489, saving model to 1b_20240723_1940_1.hdf5
150/150 [==============================] - 9s 25ms/step - loss: 0.2949 - mse: 0.2949 - mae: 0.3689 - mape: 212.9063 - rmse: 0.5430 - val_loss: 0.6682 - val_mse: 0.6682 - val_mae: 0.5250 - val_mape: 179.0792 - val_rmse: 0.8174 - lr: 0.0010
Epoch 2/15
148/150 [============================>.] - ETA: 0s - loss: 0.0424 - mse: 0.0424 - mae: 0.1632 - mape: 92.9157 - rmse: 0.2060
Epoch 2: loss improved from 0.29489 to 0.04246, saving model to 1b_20240723_1940_1.hdf5
150/150 [==============================] - 3s 23ms/step - loss: 0.0425 - mse: 0.0425 - mae: 0.1631 - mape: 92.7288 - rmse: 0.2061 - val_loss: 0.6272 - val_mse: 0.6272 - val_mae: 0.5457 - val_mape: 238.3592 - val_rmse: 0.7919 - lr: 0.0010
Epoch 3/15
148/150 [=============

,RRU.PrbUsedDl_predict,RRU.PrbUsedUl_predict,RRU.PrbUsedDl,RRU.PrbUsedUl
datetime_column,,,,
2022-12-31 18:00:00,98.635051,98.631946,98.333333,98.333333
2022-12-31 18:01:00,100.767970,100.622541,100.000000,100.000000
2022-12-31 18:02:00,74.324838,74.133209,75.083333,75.083333
2022-12-31 18:03:00,101.300761,102.117608,100.000000,100.000000
2022-12-31 18:04:00,99.693884,99.912147,100.000000,100.000000
...,...,...,...,...
2023-01-06 23:55:00,99.514372,99.148589,98.066667,98.066667
2023-01-06 23:56:00,97.722986,97.477762,97.233333,97.233333
2023-01-06 23:57:00,94.255888,94.634701,93.816667,93.816667


MAE: 0.4380639091436318, MAPE %: 0.5349278144523122, MSE: 0.30834318692876816, RMSE: 0.5552865809010409

Validation Metrics:
1/1 [==============================] - 0s 44ms/step


,RRU.PrbUsedDl_predict,RRU.PrbUsedUl_predict,RRU.PrbUsedDl,RRU.PrbUsedUl
datetime_column,,,,
2023-01-07 02:00:00,91.292561,91.601967,93.950000,93.950000
2023-01-07 02:01:00,98.120693,98.073127,100.000000,100.000000
2023-01-07 02:02:00,97.001153,97.348718,96.650000,96.650000
2023-01-07 02:03:00,82.760322,82.727749,72.933333,72.933333
2023-01-07 02:04:00,97.045844,97.451540,100.000000,100.000000
...,...,...,...,...
2023-01-07 15:55:00,90.659097,91.518532,63.333333,63.333333
2023-01-07 15:56:00,90.385862,90.378207,100.000000,100.000000
2023-01-07 15:57:00,92.086500,91.570050,100.000000,100.000000


MAE: 8.040473439119875, MAPE %: 11.219877449205368, MSE: 130.9094667859304, RMSE: 11.441567496891778
1b_20240723_1940_1.hdf5
Cell Name:  S7/B2/C1
file before:  1b_20240723_1940_1.hdf5
Epoch 1/15
150/150 [==============================] - ETA: 0s - loss: 0.2436 - mse: 0.2436 - mae: 0.3618 - mape: 122.1650 - rmse: 0.4936
Epoch 1: loss improved from inf to 0.24361, saving model to 1b_20240723_1940_2.hdf5
150/150 [==============================] - 5s 25ms/step - loss: 0.2436 - mse: 0.2436 - mae: 0.3618 - mape: 122.1650 - rmse: 0.4936 - val_loss: 0.5409 - val_mse: 0.5409 - val_mae: 0.6155 - val_mape: 201.2374 - val_rmse: 0.7355 - lr: 0.0010
Epoch 2/15
148/150 [============================>.] - ETA: 0s - loss: 0.0373 - mse: 0.0373 - mae: 0.1537 - mape: 56.9980 - rmse: 0.1932
Epoch 2: loss improved from 0.24361 to 0.03738, saving model to 1b_20240723_1940_2.hdf5
150/150 [==============================] - 3s 23ms/step - loss: 0.0374 - mse: 0.0374 - mae: 0.1536 - mape: 56.9580 - rmse: 0.1934 - 

...

,RRU.PrbUsedDl_predict,RRU.PrbUsedUl_predict,RRU.PrbUsedDl,RRU.PrbUsedUl
datetime_column,,,,
2022-12-31 18:00:00,8.061599,8.061582,7.483333,7.483333
2022-12-31 18:01:00,23.403517,23.403534,22.483333,22.483333
2022-12-31 18:02:00,1.087998,1.088041,0.000000,0.000000
2022-12-31 18:03:00,-0.390938,-0.390923,0.000000,0.000000
2022-12-31 18:04:00,-0.219785,-0.219802,0.000000,0.000000
...,...,...,...,...
2023-01-06 23:55:00,14.575127,14.575106,13.116667,13.116667
2023-01-06 23:56:00,6.093638,6.093618,5.533333,5.533333
2023-01-06 23:57:00,-0.085601,-0.085606,0.000000,0.000000


MAE: 0.5791804333411383, MAPE %: inf, MSE: 0.5326460417831429, RMSE: 0.7298260352872751

Validation Metrics:
1/1 [==============================] - 0s 26ms/step


,RRU.PrbUsedDl_predict,RRU.PrbUsedUl_predict,RRU.PrbUsedDl,RRU.PrbUsedUl
datetime_column,,,,
2023-01-07 02:00:00,3.857733,3.857995,0.000000,0.000000
2023-01-07 02:01:00,9.777266,9.777266,7.500000,7.500000
2023-01-07 02:02:00,17.818340,17.818336,3.750000,3.750000
2023-01-07 02:03:00,12.290502,12.290647,2.500000,2.500000
2023-01-07 02:04:00,16.531648,16.533609,16.233333,16.233333
...,...,...,...,...
2023-01-07 15:55:00,22.870666,22.871574,27.500000,27.500000
2023-01-07 15:56:00,11.304953,11.305039,4.000000,4.000000
2023-01-07 15:57:00,15.934490,15.934454,32.166667,32.166667


MAE: 8.868188340544052, MAPE %: inf, MSE: 150.6086426060192, RMSE: 12.272271289619505
1b_20240723_1940_29.hdf5
Cell Name:  S9/B13/C2
file before:  1b_20240723_1940_29.hdf5
Epoch 1/15
148/150 [============================>.] - ETA: 0s - loss: nan - mse: nan - mae: nan - mape: nan - rmse: nan
Epoch 1: loss did not improve from inf
150/150 [==============================] - 4s 24ms/step - loss: nan - mse: nan - mae: nan - mape: nan - rmse: nan - val_loss: nan - val_mse: nan - val_mae: nan - val_mape: nan - val_rmse: nan - lr: 0.0010
Epoch 2/15
148/150 [============================>.] - ETA: 0s - loss: nan - mse: nan - mae: nan - mape: nan - rmse: nan
Epoch 2: loss did not improve from inf

Epoch 2: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
150/150 [==============================] - 3s 23ms/step - loss: nan - mse: nan - mae: nan - mape: nan - rmse: nan - val_loss: nan - val_mse: nan - val_mae: nan - val_mape: nan - val_rmse: nan - lr: 0.0010
Epoch 3/15
149/150 [===

OSError: No file or directory found at 1b_20240723_1940_30.hdf5